In [1]:
import pandas as pd

In [35]:
df = pd.DataFrame(columns=['label', 'water level', 'velocity', 'M2 amp', 'M2 phase', 'M4 amp', 'M4 phase'])

In [44]:
df['water level'] = [1,2,3]
df['velocity'] = [1,2,3]
df

,label,water level,velocity,M2 amp,M2 phase,M4 amp,M4 phase
0,NaN,1,1,NaN,NaN,NaN,NaN
1,NaN,2,2,NaN,NaN,NaN,NaN
2,NaN,3,3,NaN,NaN,NaN,NaN
